# Local Hyper Parameter Search with Optuna


In this notebook, we'll use [Optuna](https://optuna.org) for automatic hyperparameter search to optimize our model for a machine learning task.

We will go through the definition of an objective function, the creation an Optuna study, and the launching of the optimization process to find the best hyperparameters.

Parallelization concepts will be shown in the next tutorials but rely on this simple sequential search.


### Step 1: Prepare your data 

In [1]:
import numpy as np
import reservoirpy as rpy

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import doublescroll
from reservoirpy.observables import nrmse

First, we download our dataset and divide it into a training and a testing set.

We will use them in the following step of the tutorial.

In [2]:
# total number of timesteps for train and test
timesteps = 2000

# generation of the dataset based on the Double Scroll task given some initial conditions
x0 = [0.37926545, 0.058339, -0.08167691]
X = doublescroll(timesteps, x0=x0, method="RK23")

# number of timesteps for training
train_len = 1000

# spliting dataset in train and test set for the next step
X_train = X[:train_len]
y_train = X[1 : train_len + 1]

X_test = X[train_len : -1]
y_test = X[train_len + 1:]

dataset = ((X_train, y_train), (X_test, y_test))

### Step 2: Define fixed parameters for the hyper parameter search

In [3]:
import optuna
from optuna.storages import JournalStorage, JournalFileStorage

# No need to be too verbose here
optuna.logging.set_verbosity(optuna.logging.ERROR)
rpy.verbosity(0)

0

In this section of the tutorial, we will conduct an Optuna hyperparameter search through multiple trials. Each trial will involve simulating a model with a specific configuration of hyperparameters. The results of each simulation will be evaluated and stored in a database, which can be implemented using SQL or JournalStorage for example.

Before starting the optimization process, we define the fixed hyperparameter values we want to use, such as the nomber of reservoir units (`units`), the input scaling (`iss`), and the regularization parameter (`ridge`) for the reservoir part.  we are going to use to evaluate the performance of the hyperparameters during a trial. 

We also define the number of random seeds (`nb_seeds`). In a trial, we use different random seeds to ensure that our evaluation of the model's performance is robust. Think of each seed as a starting point for the model's training process. By using multiple seeds, we're essentially training and evaluating the model multiple times with slightly different initial conditions for our reservoir. For example, if we set this value to 3, it means we'll perform the trial three times, each with a different random seed. This results in three separate models, each trained on a unique randomly generated reservoir. The final performance of the trial is then determined by taking the average of the performance scores obtained from these three individual runs.

In [4]:
# Trial Fixed hyper-parameters
nb_seeds = 3
N = 500
iss = 0.9
ridge = 1e-7

### Step 3: Define an Objective Function

In Optuna, an objective function is a user-defined function that evaluates the performance of a model using a specific set of hyperparameters. 
The main goal of the objective function is to determine the model's performance metric, which Optuna aims to either maximize or minimize during the hyperparameter search.
The objective function serves as a bridge between the optimization process and the model evaluation, allowing Optuna to iteratively explore different hyperparameter configurations to find the optimal combination that maximizes or minimizes the performance metric.

It isn't shown here but optuna also supports [multi-objective optimizations](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/002_multi_objective.html).




In [5]:
def objective(trial):
    # Record objective value(s) for each trial
    losses = []

    # Trial generated parameters (with log scale)
    sr = trial.suggest_float("sr_1", 1e-2, 10, log=True)
    lr = trial.suggest_float("lr_1", 1e-3, 1, log=True)

    for seed in range(nb_seeds):
        reservoir = Reservoir(N,
                              sr=sr,
                              lr=lr,
                              input_scaling=iss,
                              seed=seed)
        
        readout = Ridge(ridge=ridge)

        model = reservoir >> readout

        # Train and test your model
        predictions = model.fit(X_train, y_train).run(X_test)

        # Compute the desired metrics to evaluate it
        loss = nrmse(y_test, predictions, norm_value=np.ptp(X_train))

        losses.append(loss)

    return np.mean(losses)

### Step 4: Create an Optuna Study

A study is a tool to automatically search for the best hyperparameters for a model. It explores different combinations of hyperparameter values to find the configuration that maximizes the model's performance.

To create one, we need to specify a sampling method (RandomSampler(), TPESampler() ...) for exploring hyperparameter space. Then, we choose a storage backend (SQLite, JournalStorage ...) to store trials information. Finally Optuna manages the hyperparameter search process, enabling efficient exploration and optimization.

In [6]:
# Define study parameters
nb_trials = 100

sampler = optuna.samplers.RandomSampler() 

study_name = 'optuna_tutorial'
log_name = f"optuna-journal_{study_name}.log"

storage = optuna.storages.RDBStorage(f'sqlite:///{log_name}.db')

In [7]:
# Create study
study = optuna.create_study(
    study_name=study_name,
    direction="minimize",
    sampler=sampler,
    storage=storage,
    load_if_exists=True)


### Step 5: Launch Optimization

This might take some time... You can skip this step and see how to load a study and visualize the results in the next step.

In [8]:
# Optimize study with Objective function
study.optimize(objective, n_trials=nb_trials)

### Step 6: Visualize the results 

Optuna provides intuitive [visualizations](https://optuna.readthedocs.io/en/stable/reference/visualization/index.html) for hyperparameter optimization, allowing easy analysis and understanding of the search process and results.

Here are two example of plots you can draw but there are many more !

In [12]:
from optuna.visualization import plot_slice
from optuna.visualization import plot_contour

If you launched the hyper parameter search in another directory or on a remote cluster, you can easily load the study with the associated storage with the following code. 

In [13]:
study = optuna.load_study(
    study_name = f'{study_name}',
    storage = storage
)

When your study is loaded, you can then apply the visualization function of your choice.

In [14]:
plot_slice(study)

In [15]:
plot_contour(study)